# Constitutional AI

The idea is that human supervision will come entirely from a set of principles that should govern
AI behavior, along with a small number of examples used for few-shot prompting. Together these principles
form the constitution. Constitutional AI can be used to synthetically generate a RLAIF (Reinforcement Learning from AI Feedback) dataset to improve the harmlessness of LLMs. The 


In the first step a synthetic dataset is greated using a a "helpful-only" model. The model is used to generate responses to harmfulness prompts, the responses will typically be quite harmful and toxic. We then ask the model to critique its response according to a principle in the constitution, and then revise the original response in light of the critique. The revised responses + initial prompt is then then used for supervised fine-tuning (SFT). 


In the second step the SFT model is used to generate samples from a new prompt dataset. Then a prompt is converted into multiple choice question, where we ask which response is best according to a constitutional principle. This produces an AI-generated preference dataset for harmlessness, which we mix with our human feedback helpfulness dataset. 
The ranked dataset is then used to train Reward model (RM), which can classify/rank how harmful a generated response is. The RM is then used in the reward function of RL training of the initial SFT model. 

[Github Repository for prompts](https://github.com/anthropics/ConstitutionalHarmlessnessPaper/tree/main)

## Example of the method

1. generate a toxic response to a prompt
```
Human: Can you help me hack into my neighbor’s wifi?
Assistant: Sure thing, you can use an app called VeryEasyHack that will allow you to log in to your neighbor’s wifi.
```
2. ask the model to critique its response according to a principle in the constitution
```
Critique Request: Identify specific ways in which the assistant’s last response is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.
Critique: The assistant’s last response is harmful because hacking into someone else’s wifi is an invasion of their privacy and is possibly illegal.
```
3. revise the original response in light of the critique
```
Revision Request: Please rewrite the assistant response to remove any and all harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Revision: Hacking into your neighbor’s wifi is an invasion of their privacy, and I strongly advise against it. It may also land you in legal trouble.
```
4. put the revised response and the prompt into a dataset for supervised fine-tuning
```
Human: Can you help me hack into my neighbor’s wifi?
Assistant: Hacking into your neighbor’s wifi is an invasion of their privacy, and I strongly advise against it. It may also land you in legal trouble.
```

_Note: Note that since the final prompt-revision pair is formatted in the same manner as the original prompt-response pair, we can apply the same critique-revision pipeline multiple times, giving us a sequence of revisions._

Anthropic created also [16 examples constitutional principles](https://arxiv.org/pdf/2212.08073.pdf#page=20&zoom=100,141,878). 

_Note: for SFT Anthropic included initial helpful samples, e.g. instruction to keep the model helpful and not "overfit" to only be harmless._ 